In [1]:
#will actually need this
#!pip install librosa

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import scipy
import matplotlib.pyplot as plt
import librosa
#from pyAudioAnalysis import audioBasicIO #don't really need pyaudioanalysis
#from pyAudioAnalysis import ShortTermFeatures

covariance:

6 combinations:
X X
X Y
X Z
Y Y
Z Z
Y Z

#### Run all below until next markdown

In [43]:
#must run join_together notebook to generate joined.pkl
df = pd.read_pickle('joined.pkl')
ids = df.pid.unique()

In [5]:
def getZeroCrossingRate(arr):
        my_array = np.array(arr)
        return float("{0:.2f}".format((((my_array[:-1] * my_array[1:]) < 0).sum())/len(arr)))

In [6]:
def fft_and_zero_cross(df, pid):
    
    sec = 51
    short = sec*2
    full = sec*10
    big = sec*1000
    
    df = df[df['pid'] == pid].copy().reset_index().drop('index', axis = 1)
    
    windows = np.arange(0,len(df),big)
    
    
    df['x_fft_1'] = 0
    df['x_fft_2'] = 0
    df['x_fft_3'] = 0

    df['y_fft_1'] = 0
    df['y_fft_2'] = 0
    df['y_fft_3'] = 0
    
    df['z_fft_1'] = 0
    df['z_fft_2'] = 0
    df['z_fft_3'] = 0

    df_app = pd.DataFrame(columns = df.columns)
    
    for i in range(len(windows)-1):
        
        small = df.iloc[windows[i]:windows[i+1]].copy()
        
        x_fft = scipy.fft.fft(np.array(small['x'])).real
        
        small['x_fft_1'] = x_fft[0]
        small['x_fft_2'] = x_fft[1]
        small['x_fft_3'] = x_fft[2]
        small['x_fft_max'] = x_fft.max()
        
        small['x_zcr'] = getZeroCrossingRate(np.array(small['x']))
        
        y_fft = scipy.fft.fft(np.array(small['y'])).real
        
        small['y_fft_1'] = y_fft[0]
        small['y_fft_2'] = y_fft[1]
        small['y_fft_3'] = y_fft[2]
        small['y_fft_max'] = y_fft.max()
        
        small['y_zcr'] = getZeroCrossingRate(np.array(small['y']))
        
        z_fft = scipy.fft.fft(np.array(small['z'])).real
        
        small['z_fft_1'] = z_fft[0]
        small['z_fft_2'] = z_fft[1]
        small['z_fft_3'] = z_fft[2]
        small['z_fft_max'] = z_fft.max()
        
        small['z_zcr'] = getZeroCrossingRate(np.array(small['z']))
        
        df_app = df_app.append(small)
        
    df = df_app.reset_index().drop('index', axis = 1)
    
    return df

In [7]:
def mfcc(df, pid):
    
    #give whole df, one pid at a time
    
    df = df[df['pid'] == pid].copy().reset_index().drop('index', axis = 1)
    
    Fs = 40
    mfcc_x_array = librosa.feature.mfcc(np.array(df['x']), sr=Fs).T
    mfcc_y_array = librosa.feature.mfcc(np.array(df['y']), sr=Fs).T
    mfcc_z_array = librosa.feature.mfcc(np.array(df['z']), sr=Fs).T

    x_mfcc = pd.DataFrame(mfcc_x_array)
    y_mfcc = pd.DataFrame(mfcc_y_array)
    z_mfcc = pd.DataFrame(mfcc_z_array)

    expansion_x = round(len(df)/len(x_mfcc))
    new_index_x = (pd.DataFrame(x_mfcc).index * expansion_x)
    x_mfcc.index = new_index_x

    expansion_y = round(len(df)/len(y_mfcc))
    new_index_y = (pd.DataFrame(y_mfcc).index * expansion_y)
    y_mfcc.index = new_index_x

    expansion_z = round(len(df)/len(z_mfcc))
    new_index_z = (pd.DataFrame(z_mfcc).index * expansion_z)
    z_mfcc.index = new_index_z

    df = pd.merge_asof(df,x_mfcc, left_index = True, right_index = True)
    df = pd.merge_asof(df,y_mfcc, left_index = True, right_index = True)
    df = pd.merge_asof(df,z_mfcc, left_index = True, right_index = True)
    
    df = df.reset_index().drop('index', axis = 1)
 
    #new_names = [(i,i+'_z') for i in df.iloc[:, 6:].columns.values]
    #df.rename(columns = dict(new_names), inplace=True)

    return df

In [8]:
def rms(df,pid):
    
    #give whole df, one pid at a time
    
    df = df[df['pid'] == pid].copy()
    
    sec = 51
    short = sec*2
    sec5 = sec*5
    full = sec*10
    
    df['x_rms'] = (df['x']**2).rolling(window=short).mean()**.5
    df['y_rms'] = (df['y']**2).rolling(window=short).mean()**.5
    df['z_rms'] = (df['z']**2).rolling(window=short).mean()**.5
    
    df['x_rms_diff_sec'] = df['x_rms'].diff(periods = sec)
    df['y_rms_diff_sec'] = df['y_rms'].diff(periods = sec)
    df['z_rms_diff_sec'] = df['z_rms'].diff(periods = sec)
    
    df['x_rms_diff_sec5'] = df['x_rms'].diff(periods = sec5)
    df['y_rms_diff_sec5'] = df['y_rms'].diff(periods = sec5)
    df['z_rms_diff_sec5'] = df['z_rms'].diff(periods = sec5)
    
    df = df.reset_index().drop('index', axis = 1)
    
    return df

In [9]:
def basic(df,pid):
     
    sec = 51
    short = sec*2
    full = sec*10
    
    df = df[df['pid'] == pid].copy()
    
    df['x_mean_short'] = df['x'].rolling(window=short).mean()
    df['x_std_short'] = df['x'].rolling(window=short).std()
    df['x_mean_long'] = df['x'].rolling(window=full).mean()
    df['x_std_long'] = df['x'].rolling(window=full).std()
    
    df['y_mean_short'] = df['y'].rolling(window=short).mean()
    df['y_std_short'] = df['y'].rolling(window=short).std()
    df['y_mean_long'] = df['y'].rolling(window=full).mean()
    df['y_std_long'] = df['y'].rolling(window=full).std()
                          
    df['z_mean_short'] = df['z'].rolling(window=short).mean()
    df['z_std_short'] = df['z'].rolling(window=short).std()
    df['z_mean_long'] = df['z'].rolling(window=full).mean()
    df['z_std_long'] = df['z'].rolling(window=full).std()
    
    df = df.reset_index().drop('index', axis = 1)
    
    return df

#### Choose one of the five methods below, then pickle.

In [10]:
#1

#apply each function (without extract or summarize) and take every 25 rows (~half second) for each user

start = datetime.now()

#indexing this much bc fft and zero cross window is 1000 seconds
cold = fft_and_zero_cross(df.iloc[0:51100], ids[0])
cold = mfcc(cold, ids[0])
cold = rms(cold, ids[0])
cold = basic(cold,ids[0])

grab_every = pd.DataFrame(columns = cold.columns)

for i in range(len(ids)):
    temp_grab =  fft_and_zero_cross(df, ids[i])
    temp_grab = mfcc(temp_grab, ids[i])
    temp_grab = rms(temp_grab, ids[i])
    temp_grab = basic(temp_grab,ids[i])
    temp_grab = temp_grab.iloc[range(0,len(temp_grab),25)]
    grab_every = grab_every.append(temp_grab)

df = grab_every.dropna(axis = 0)
    
end = datetime.now()

end - start

datetime.timedelta(seconds=278, microseconds=273121)

In [11]:
df.head()

,time,pid,x,y,z,TAC_Reading,x_fft_1,x_fft_2,x_fft_3,y_fft_1,...,x_mean_long,x_std_long,y_mean_short,y_std_short,y_mean_long,y_std_long,z_mean_short,z_std_short,z_mean_long,z_std_long
525,1493739742587,JB3156,-0.006592,0.005908,0.008528,0.035549,5469.112931,-4486.878527,2775.130233,-21708.270315,...,-0.056394,0.184828,0.000558,0.020346,0.118906,0.665301,0.024413,0.012756,0.110147,0.302441
550,1493739743089,JB3156,-0.008537,-0.019438,0.001342,0.035549,5469.112931,-4486.878527,2775.130233,-21708.270315,...,-0.028135,0.025008,-0.000059,0.017398,0.017216,0.055145,0.023044,0.016047,0.076599,0.064116
575,1493739743593,JB3156,-0.008419,0.041303,0.000147,0.035549,5469.112931,-4486.878527,2775.130233,-21708.270315,...,-0.025369,0.024102,-0.000554,0.020511,0.007618,0.036192,0.016335,0.016577,0.073558,0.065936
600,1493739744100,JB3156,-0.008186,0.026511,-0.002250,0.035549,5469.112931,-4486.878527,2775.130233,-21708.270315,...,-0.024281,0.023368,-0.000320,0.022740,0.003794,0.027094,0.008562,0.014980,0.071013,0.067481
625,1493739744604,JB3156,0.026681,-0.003895,0.006128,0.035549,5469.112931,-4486.878527,2775.130233,-21708.270315,...,-0.023721,0.023702,0.000016,0.023791,0.001995,0.024722,0.003385,0.013997,0.069401,0.068787


In [ ]:
#2

#apply each function (without extract or summarize) and take every 510 rows (~10 seconds) for each user

start = datetime.now()

#indexing this much bc fft and zero cross window is 1000 seconds
cold = fft_and_zero_cross(df.iloc[0:51100], ids[0])
cold = mfcc(cold, ids[0])
cold = rms(cold, ids[0])
cold = basic(cold,ids[0])

grab_every = pd.DataFrame(columns = cold.columns)

for i in range(len(ids)):
    temp_grab =  fft_and_zero_cross(df, ids[i])
    temp_grab = mfcc(temp_grab, ids[i])
    temp_grab = rms(temp_grab, ids[i])
    temp_grab = basic(temp_grab,ids[i])
    temp_grab = temp_grab.iloc[range(0,len(temp_grab),510)]
    grab_every = grab_every.append(temp_grab)

df = grab_every.dropna(axis = 0)
    
end = datetime.now()

end - start

In [44]:
#3

#apply each function (without extract or summarize) and collapse to mean for every 25 rows (~half second) for each user

start = datetime.now()

#indexing this much bc fft and zero cross window is 1000 seconds
cold = fft_and_zero_cross(df.iloc[0:51100], ids[0])
cold = mfcc(cold, ids[0])
cold = rms(cold, ids[0])
cold = basic(cold,ids[0])

grab_every = pd.DataFrame(columns = cold.columns)

for i in range(len(ids)):
    temp_grab =  fft_and_zero_cross(df, ids[i])
    temp_grab = mfcc(temp_grab, ids[i])
    temp_grab = rms(temp_grab, ids[i])
    temp_grab = basic(temp_grab,ids[i])
    sep = temp_grab
    temp_grab = temp_grab.rolling(window=25).mean()
    temp_grab[['time','pid']] = sep[['time','pid']]
    temp_grab = temp_grab.iloc[range(0,len(temp_grab),25)]
    grab_every = grab_every.append(temp_grab)

df = grab_every.dropna(axis = 0)
    
end = datetime.now()

end - start

datetime.timedelta(seconds=373, microseconds=297147)

In [45]:
df.head()

,time,pid,x,y,z,TAC_Reading,x_fft_1,x_fft_2,x_fft_3,y_fft_1,...,x_mean_long,x_std_long,y_mean_short,y_std_short,y_mean_long,y_std_long,z_mean_short,z_std_short,z_mean_long,z_std_long
550,1493739743089,JB3156,-0.001264,-0.000676,0.010298,0.035549,5469.112931,-4486.878527,2775.130233,-21708.270315,...,-0.034094,0.049120,0.000564,0.018630,0.039108,0.162378,0.023960,0.014182,0.080859,0.076736
575,1493739743593,JB3156,-0.000200,-0.001266,0.003833,0.035549,5469.112931,-4486.878527,2775.130233,-21708.270315,...,-0.026419,0.024157,0.000143,0.017348,0.011572,0.043020,0.019887,0.016721,0.075035,0.065067
600,1493739744100,JB3156,-0.000568,0.000356,0.000433,0.035549,5469.112931,-4486.878527,2775.130233,-21708.270315,...,-0.024696,0.024047,-0.000213,0.022140,0.005563,0.032051,0.012402,0.016047,0.072221,0.066789
625,1493739744604,JB3156,0.000053,0.001275,-0.001437,0.035549,5469.112931,-4486.878527,2775.130233,-21708.270315,...,-0.024029,0.023464,-0.000479,0.023460,0.002772,0.025805,0.005521,0.014418,0.070151,0.068187
650,1493739745108,JB3156,0.003612,0.003930,-0.001821,0.035549,5469.112931,-4486.878527,2775.130233,-21708.270315,...,-0.023274,0.024155,0.000297,0.024232,0.001429,0.024594,0.001859,0.012854,0.067886,0.069603


In [ ]:
#4

#apply each function (without extract or summarize) and collapse to mean for every 510 rows (~10 seconds) for each user

start = datetime.now()

#indexing this much bc fft and zero cross window is 1000 seconds
cold = fft_and_zero_cross(df.iloc[0:51100], ids[0])
cold = mfcc(cold, ids[0])
cold = rms(cold, ids[0])
cold = basic(cold,ids[0])

grab_every = pd.DataFrame(columns = cold.columns)

for i in range(len(ids)):
    temp_grab =  fft_and_zero_cross(df, ids[i])
    temp_grab = mfcc(temp_grab, ids[i])
    temp_grab = rms(temp_grab, ids[i])
    temp_grab = basic(temp_grab,ids[i])
    sep = temp_grab
    temp_grab = temp_grab.rolling(window=510).mean()
    temp_grab[['time','pid']] = sep[['time','pid']]
    temp_grab = temp_grab.iloc[range(0,len(temp_grab),25)]
    grab_every = grab_every.append(temp_grab)

df = grab_every.dropna(axis = 0)
    
end = datetime.now()

end - start

In [ ]:
#5

#apply each function (without extract or summarize) and keep all rows

start = datetime.now()

# ~66 seconds
fz_df = pd.DataFrame(columns = (fft_and_zero_cross(df, ids[0])).columns)

for i in range(len(ids)):
    fz_df = fz_df.append(fft_and_zero_cross(df,ids[i]))
    
df = fz_df

print('completed fft and zero cross')

# ~90 seconds
mfcc_df = pd.DataFrame(columns = (mfcc(df, ids[0])).columns)

for i in range(len(ids)):
    mfcc_df = mfcc_df.append(mfcc(df,ids[i]))
    
df = mfcc_df

print('completed mfcc')

# ~156 seconds
rms_df = pd.DataFrame(columns = (rms(df, ids[0])).columns)

for i in range(len(ids)):
    rms_df = rms_df.append(rms(df,ids[i]))
    
df = rms_df

print('completed rms')

# ~298 seconds
basic_df = pd.DataFrame(columns = (basic(df, ids[0])).columns)

for i in range(len(ids)):
    basic_df = basic_df.append(basic(df,ids[i]))
    
df = basic_df

print('completed rolling means and stds')

df = df.dropna(axis = 0)

end = datetime.now()

end - start

In [46]:
#uncollapsed is ~8 gigs
#half second version is ~440 megs

start = datetime.now()

df.to_pickle('./processed.pkl')

end = datetime.now()

end - start

datetime.timedelta(seconds=1, microseconds=2224)